In [ ]:
import face_recognition
import cv2
import numpy as np
import csv
import os
from datetime import datetime
from tkinter import *
from tkinter import messagebox


In [ ]:
video_capture = cv2.VideoCapture(0)

known_faces = {
    "JAY": face_recognition.face_encodings(face_recognition.load_image_file("JAY.jpg"))[0],
    "AMAN": face_recognition.face_encodings(face_recognition.load_image_file("AMAN.jpg"))[0],
    "AYUSHI": face_recognition.face_encodings(face_recognition.load_image_file("AYUSHI.jpg"))[0],
    "SANAYA": face_recognition.face_encodings(face_recognition.load_image_file("SANAYA.jpg"))[0]
}

known_face_encodings = list(known_faces.values())
known_face_names = list(known_faces.keys())

students = known_face_names.copy()

In [ ]:
face_locations = []
face_encodings = []
face_names = []
s = True

now = datetime.now()
current_date = now.strftime("%Y-%m-%d")

with open(current_date + '.csv', 'w+', newline='') as f:
    lnwriter = csv.writer(f)

In [ ]:
def start_recognition():
    global s
    s = True

def stop_recognition():
    global s
    s = False

root = Tk()
root.title("Facial Recognition Attendance System")

start_button = Button(root, text="Start", command=start_recognition)
start_button.pack(pady=10)

stop_button = Button(root, text="Stop", command=stop_recognition)
stop_button.pack(pady=10)

root.geometry("300x200")

In [ ]:
while True:
    _, frame = video_capture.read()
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    rgb_small_frame = small_frame[:, :, ::-1]

    if s:
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
        face_names = []
        
        for face_encoding in face_encodings:
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = ""
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]
            face_names.append(name)
            
            if name in known_face_names:
                font = cv2.FONT_HERSHEY_SIMPLEX
                cv2.putText(frame, name + ' Present', (10, 100), font, 1.5, (255, 0, 0), 3, 2)

                if name in students:
                    students.remove(name)
                    current_time = now.strftime("%H-%M-%S")
                    lnwriter.writerow([name, current_time])

    cv2.imshow("Attendance System", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

In [ ]:
root.mainloop()